### 20240822_Day 4

In [9]:
import os
import pickle
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet50, ResNet50_Weights, resnet18, ResNet18_Weights
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
## Load your model any way you want; it should be stand-alone
## Call it `new_model`

file_name = '/content/drive/MyDrive/students_submission/model_20Aug.pkl'
with open(file_name, 'rb') as f:
    new_model = pickle.load(f)

In [10]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

class RandomDataset(Dataset):
    def __init__(self):
        self.num_classes = 5
        self.num_images_per_class = 20
        self.image_size = (3, 224, 224)
        self.transform = transforms.Compose([
            transforms.Resize(size=(224, 224)),
            transforms.Normalize(mean=mean, std=std)
        ])
        self.total_images = self.num_classes * self.num_images_per_class

    def __len__(self):
        return self.total_images

    def __getitem__(self, idx):
        image = np.random.rand(*self.image_size).astype(np.float32)
        label = idx // self.num_images_per_class
        image = torch.tensor(image)
        image = self.transform(image)
        return image, label


dataloader = DataLoader(
    RandomDataset(), batch_size=8, shuffle=False
)

In [ ]:
device = torch.device(
    torch.cuda.current_device() if torch.cuda.is_available() else 'cpu'
)

holder = {}
holder['y_true'] = []
holder['y_hat'] = []

new_model = new_model.to(device)

new_model.eval()
with torch.no_grad():
    for data in dataloader:
        images, y_true = data
        images, y_true = images.to(device), y_true.to(device)

        outputs = new_model(images)
        _, y_hat = torch.max(outputs, 1)   # logits not required, index pos is sufficient
        holder['y_true'].extend(
            list(y_true.cpu().detach().numpy())
        )
        holder['y_hat'].extend(
            list(y_hat.cpu().detach().numpy())
        )

y_true_all = holder['y_true']
y_pred_all = holder['y_hat']
M = confusion_matrix(y_true_all, y_pred_all)
print("Confusion matrix: \n", M)

C = np.array([
    [0, 2, 5, 3, 4],
    [2, 0, 6, 4, 3],
    [5, 6, 0, 1, 2],
    [3, 4, 1, 0, 5],
    [4, 3, 2, 5, 0]
])
average_cost = np.sum(M*C)/np.sum(M)
print(average_cost)